In [ ]:
# Стандартные библиотеки
import random # библиотека функций для генерации случайных значений
# Сторонние библиотеки
import numpy as np # библиотека функций для работы с матрицами
import gzip # библиотека для сжатия и распаковки файлов gzip и gunzip.
import pickle # библиотека для сохранения и загрузки сложных объектов Python.
import numpy as np # библиотека для работы с матрицами

In [ ]:
def sigmoid(z): # определение сигмоидальной функции активации
    return 1.0/(1.0+np.exp(-z))

def sigmoid_prime(z):# Производная сигмоидальной функции
 return sigmoid(z)*(1-sigmoid(z))

def load_data():
    f = gzip.open('mnist.pkl.gz', 'rb') # открываем сжатый файл gzip вдвоичном режиме
    training_data, validation_data, test_data = pickle.load(f, encoding='latin1') # загружам таблицы из файла

    #training_data,  test_data = pickle.load(f, encoding='latin1')  # загружам таблицы из файла
    f.close() # закрываем файл
    return (training_data, validation_data, test_data)
    #return (training_data,  test_data)


def load_data_wrapper():
    tr_d, va_d, te_d = load_data() # инициализация наборов данных в формате MNIST
    #tr_d,  te_d = load_data()  # инициализация наборов данных в формате MNIST
    training_inputs = [np.reshape(x, (784, 1)) for x in tr_d[0]] # преобразование массивов размера 1 на 784 к массивам размера 784 на 1
    training_results = [vectorized_result(y) for y in tr_d[1]] # представление цифр от 0 до 9 в виде массивов размера 10 на 1
    training_data = zip(training_inputs, training_results) # формируем набор обучающих данных из пар (x, y)
    validation_inputs = [np.reshape(x, (784, 1)) for x in va_d[0]] # преобразование массивов размера 1 на 784 к массивам размера 784 на 1
    validation_data = zip(validation_inputs, va_d[1]) # формируем набор данных проверки из пар (x, y)
    test_inputs = [np.reshape(x, (784, 1)) for x in te_d[0]] # преобразование массивов размера 1 на 784 к массивам размера 784 на 1
    test_data = zip(test_inputs, te_d[1]) # формируем набор тестовых данных из пар (x, y)
    return (training_data, validation_data, test_data)
    #return (training_data,  test_data)
def vectorized_result(j):
    e = np.zeros((10, 1))
    e[j] = 1.0
    return e

In [ ]:
class Network(object): # используется для описания нейронной сети
    def __init__(self, sizes): # конструктор класса
     # self – указатель на объект класса
     # sizes – список размеров слоев нейроннойсети
        self.num_layers = len(sizes) # задаем количество слоев нейроннойсети
        self.sizes = sizes # задаем список размеров слоев нейронной сети
        self.biases = [np.random.randn(y, 1) for y in sizes[1:]] # задаемслучайные начальные смещения
        self.weights = [np.random.randn(y, x) for x, y in zip(sizes[:-1],sizes[1:])] # задаем случайные начальные веса связей

    def feedforward(self, a):
        for b, w in zip(self.biases, self.weights):
            a = sigmoid(np.dot(w, a) + b)
        return a
    def SGD( # Стохастический градиентный спуск
         self # указатель на объект класса
         , training_data # обучающая выборка
         , epochs # количество эпох обучения
         , mini_batch_size # размер подвыборки
         , eta # скорость обучения
         , test_data # тестирующая выборка
         ):
            test_data = list(test_data) # создаем список объектов тестирующейвыборки
            n_test = len(test_data) # вычисляем длину тестирующей выборки
            training_data = list(training_data) # создаем список объектов обучающей выборки
            n = len(training_data) # вычисляем размер обучающей выборки
            for j in range(epochs): # цикл по эпохам
                random.shuffle(training_data) # перемешиваем элементы обучающей выборки
                mini_batches = [training_data[k:k+mini_batch_size] for k in range(0, n, mini_batch_size)] # создаем подвыборки
                for mini_batch in mini_batches: # цикл по подвыборкам
                    self.update_mini_batch(mini_batch, eta) # один шаг градиентного спуска
                print ("Epoch {0}: {1} / {2}".format(j, self.evaluate(test_data), n_test)) # смотрим прогресс в обучении

    def update_mini_batch( # Шаг градиентного спуска
         self # указатель на объект класса
         , mini_batch # подвыборка
         , eta # скорость обучения
         ):
             nabla_b = [np.zeros(b.shape) for b in self.biases] # список градиентов dC/db для каждого слоя (первоначально заполняются нулями)
             nabla_w = [np.zeros(w.shape) for w in self.weights] # список градиентов dC/dw для каждого слоя (первоначально заполняются нулями)
             for x, y in mini_batch:
                 delta_nabla_b, delta_nabla_w = self.backprop(x, y) # послойно вычисляем градиенты dC/db и dC/dw для текущего прецедента (x, y)
                 nabla_b = [nb+dnb for nb, dnb in zip(nabla_b, delta_nabla_b)] # суммируем градиенты dC/db для различных прецедентов текущей подвыборки
                 nabla_w = [nw+dnw for nw, dnw in zip(nabla_w, delta_nabla_w)] # суммируем градиенты dC/dw для различных прецедентов текущей подвыборки
             self.weights = [w-(eta/len(mini_batch))*nw for w, nw in zip(self.weights, nabla_w)] # обновляем все веса w нейронной сети
             self.biases = [b-(eta/len(mini_batch))*nb for b, nb in zip(self.biases, nabla_b)] # обновляем все смещения b нейронной сети

    def backprop( # Алгоритм обратного распространения
         self # указатель на объект класса
         , x # вектор входных сигналов
         , y # ожидаемый вектор выходных сигналов
         ):
         nabla_b = [np.zeros(b.shape) for b in self.biases] # списокградиентов dC/db для каждого слоя (первоначально заполняются нулями)
         nabla_w = [np.zeros(w.shape) for w in self.weights] # списокградиентов dC/dw для каждого слоя (первоначально заполняются нулями)
         # определение переменных
         activation = x # выходные сигналы слоя (первоначально соответствуетвыходным сигналам 1-го слоя или входным сигналам сети)
         activations = [x] # список выходных сигналов по всем слоям(первоначально содержит только выходные сигналы 1-го слоя)
         zs = [] # список активационных потенциалов по всем слоям(первоначально пуст)

         # прямое распространение
         for b, w in zip(self.biases, self.weights):
             z = np.dot(w, activation)+b # считаем активационные потенциалы текущего слоя
             zs.append(z) # добавляем элемент (активационные потенциалы слоя) в конец списка
             activation = sigmoid(z) # считаем выходные сигналы текущего слоя, применяя сигмоидальную функцию активации к активационным потенциалам слоя
             activations.append(activation) # добавляем элемент (выходныесигналы слоя) в конец списка

         # обратное распространение
         delta = self.cost_derivative(activations[-1], y) * sigmoid_prime(zs[-1]) # считаем меру влияния нейронов выходного слоя L навеличину ошибки (BP1)
         nabla_b[-1] = delta # градиент dC/db для слоя L (BP3)
         nabla_w[-1] = np.dot(delta, activations[-2].transpose()) # градиент dC/dw для слоя L (BP4)

         for l in range(2, self.num_layers):
             z = zs[-l] # активационные потенциалы l-го слоя (двигаемся по списку справа налево)
             sp = sigmoid_prime(z) # считаем сигмоидальную функцию от активационных потенциалов l-го слоя
             delta = np.dot(self.weights[-l+1].transpose(), delta) * sp # считаем меру влияния нейронов l-го слоя на величину ошибки (BP2)
             nabla_b[-l] = delta # градиент dC/db для l-го слоя (BP3)
             nabla_w[-l] = np.dot(delta, activations[-l-1].transpose())# градиент dC/dw для l-го слоя (BP4)
         return (nabla_b, nabla_w)

    def evaluate(self, test_data): # Оценка прогресса в обучении
         test_results = [(np.argmax(self.feedforward(x)), y)
         for (x, y) in test_data]
         return sum(int(x == y) for (x, y) in test_results)

    def cost_derivative(self, output_activations, y):    # Вычисление частных производных стоимостной функции по выходным сигналам последнего слоя
         return (output_activations-y)

In [ ]:
from datetime import datetime


training_data, validation_data, test_data = load_data_wrapper()
#training_data,  test_data = mnist_loader.load_data_wrapper()
start_time = datetime.now()

''''''

net = Network([784, 100, 10])
net.SGD(training_data, 30, 10, 3.0, test_data=test_data)

end_time = datetime.now()
print('Duration: {}'.format(end_time - start_time))

Epoch 0: 4977 / 10000
Epoch 1: 5055 / 10000
Epoch 2: 5761 / 10000
Epoch 3: 5842 / 10000
Epoch 4: 5866 / 10000
Epoch 5: 5870 / 10000
Epoch 6: 5890 / 10000
Epoch 7: 6730 / 10000
Epoch 8: 6795 / 10000
Epoch 9: 6796 / 10000
Epoch 10: 6801 / 10000
Epoch 11: 6817 / 10000
Epoch 12: 6845 / 10000
Epoch 13: 6844 / 10000
Epoch 14: 6840 / 10000
Epoch 15: 6863 / 10000
Epoch 16: 6850 / 10000
Epoch 17: 6844 / 10000
Epoch 18: 6867 / 10000
Epoch 19: 6843 / 10000
Epoch 20: 6892 / 10000
Epoch 21: 6870 / 10000
Epoch 22: 6906 / 10000
Epoch 23: 6875 / 10000
Epoch 24: 6899 / 10000
Epoch 25: 6911 / 10000
Epoch 26: 6907 / 10000
Epoch 27: 6908 / 10000
Epoch 28: 6920 / 10000
Epoch 29: 6950 / 10000
Duration: 0:06:50.212670


In [ ]:
training_data, validation_data, test_data = load_data_wrapper()
#training_data,  test_data = mnist_loader.load_data_wrapper()
start_time = datetime.now()

net = Network([784, 100, 10])
net.SGD(training_data, 30, 1000, 0.001, test_data=test_data)

end_time = datetime.now()
print('Duration: {}'.format(end_time - start_time))

Epoch 0: 778 / 10000
Epoch 1: 781 / 10000
Epoch 2: 784 / 10000
Epoch 3: 785 / 10000
Epoch 4: 789 / 10000
Epoch 5: 792 / 10000
Epoch 6: 790 / 10000
Epoch 7: 785 / 10000
Epoch 8: 787 / 10000
Epoch 9: 790 / 10000
Epoch 10: 792 / 10000
Epoch 11: 791 / 10000
Epoch 12: 795 / 10000
Epoch 13: 796 / 10000
Epoch 14: 800 / 10000
Epoch 15: 804 / 10000
Epoch 16: 806 / 10000
Epoch 17: 804 / 10000
Epoch 18: 800 / 10000
Epoch 19: 801 / 10000
Epoch 20: 802 / 10000
Epoch 21: 803 / 10000
Epoch 22: 799 / 10000
Epoch 23: 803 / 10000
Epoch 24: 804 / 10000
Epoch 25: 804 / 10000
Epoch 26: 805 / 10000
Epoch 27: 805 / 10000
Epoch 28: 806 / 10000
Epoch 29: 803 / 10000
Duration: 0:06:16.794593


In [ ]:
training_data, validation_data, test_data = load_data_wrapper()
#training_data,  test_data = mnist_loader.load_data_wrapper()
start_time = datetime.now()

net = Network([784, 30, 10])
net.SGD(training_data, 30, 10, 100.0, test_data=test_data)

end_time = datetime.now()
print('Duration: {}'.format(end_time - start_time))

Epoch 0: 1556 / 10000
Epoch 1: 1556 / 10000
Epoch 2: 1556 / 10000
Epoch 3: 1556 / 10000
Epoch 4: 1556 / 10000
Epoch 5: 1556 / 10000
Epoch 6: 1556 / 10000
Epoch 7: 1556 / 10000
Epoch 8: 1556 / 10000
Epoch 9: 1556 / 10000
Epoch 10: 1556 / 10000
Epoch 11: 1556 / 10000
Epoch 12: 1556 / 10000
Epoch 13: 1556 / 10000
Epoch 14: 1556 / 10000
Epoch 15: 1556 / 10000
Epoch 16: 1556 / 10000
Epoch 17: 1556 / 10000
Epoch 18: 1556 / 10000
Epoch 19: 1556 / 10000
Epoch 20: 1556 / 10000
Epoch 21: 1556 / 10000
Epoch 22: 1556 / 10000
Epoch 23: 1556 / 10000
Epoch 24: 1556 / 10000
Epoch 25: 1556 / 10000
Epoch 26: 1556 / 10000
Epoch 27: 1556 / 10000
Epoch 28: 1556 / 10000
Epoch 29: 1556 / 10000
Duration: 0:03:41.561315


In [ ]:
training_data, validation_data, test_data = load_data_wrapper()
#training_data,  test_data = mnist_loader.load_data_wrapper()
start_time = datetime.now()

net = Network([784, 10])
net.SGD(training_data, 30, 10, 100.0, test_data=test_data)

end_time = datetime.now()
print('Duration: {}'.format(end_time - start_time))

Epoch 0: 2275 / 10000


<ipython-input-6-d4b200e2015a>:2: RuntimeWarning: overflow encountered in exp
  return 1.0/(1.0+np.exp(-z))


Epoch 1: 2268 / 10000
Epoch 2: 2293 / 10000
Epoch 3: 2307 / 10000
Epoch 4: 2939 / 10000
Epoch 5: 2947 / 10000
Epoch 6: 2952 / 10000
Epoch 7: 3003 / 10000
Epoch 8: 3006 / 10000
Epoch 9: 2998 / 10000
Epoch 10: 2988 / 10000
Epoch 11: 3016 / 10000
Epoch 12: 2977 / 10000
Epoch 13: 3012 / 10000
Epoch 14: 2997 / 10000
Epoch 15: 3003 / 10000
Epoch 16: 2981 / 10000
Epoch 17: 2983 / 10000
Epoch 18: 3005 / 10000
Epoch 19: 3003 / 10000
Epoch 20: 2980 / 10000
Epoch 21: 2984 / 10000
Epoch 22: 2967 / 10000
Epoch 23: 3007 / 10000
Epoch 24: 3012 / 10000
Epoch 25: 2987 / 10000
Epoch 26: 2999 / 10000
Epoch 27: 2955 / 10000
Epoch 28: 2997 / 10000
Epoch 29: 3013 / 10000
Duration: 0:01:08.183151


In [ ]:
training_data, validation_data, test_data = load_data_wrapper()
#training_data,  test_data = mnist_loader.load_data_wrapper()
start_time = datetime.now()

net = Network([784, 30, 10])
net.SGD(training_data, 30, 100, 3.0, test_data=test_data)

end_time = datetime.now()
print('Duration: {}'.format(end_time - start_time))

Epoch 0: 6124 / 10000
Epoch 1: 7342 / 10000
Epoch 2: 7822 / 10000
Epoch 3: 7982 / 10000
Epoch 4: 8095 / 10000
Epoch 5: 8153 / 10000
Epoch 6: 8194 / 10000
Epoch 7: 8222 / 10000
Epoch 8: 8270 / 10000
Epoch 9: 8301 / 10000
Epoch 10: 8312 / 10000
Epoch 11: 8316 / 10000
Epoch 12: 8363 / 10000
Epoch 13: 8367 / 10000
Epoch 14: 8372 / 10000
Epoch 15: 8397 / 10000
Epoch 16: 8388 / 10000
Epoch 17: 8411 / 10000
Epoch 18: 8424 / 10000
Epoch 19: 8436 / 10000
Epoch 20: 8430 / 10000
Epoch 21: 8451 / 10000
Epoch 22: 8456 / 10000
Epoch 23: 8464 / 10000
Epoch 24: 8467 / 10000
Epoch 25: 8477 / 10000
Epoch 26: 8470 / 10000
Epoch 27: 8494 / 10000
Epoch 28: 8483 / 10000
Epoch 29: 8487 / 10000
Duration: 0:03:50.950794


In [ ]:
training_data, validation_data, test_data = load_data_wrapper()
#training_data,  test_data = mnist_loader.load_data_wrapper()
start_time = datetime.now()

net = Network([784, 30, 10])
net.SGD(training_data, 15, 100, 3.0, test_data=test_data)

end_time = datetime.now()
print('Duration: {}'.format(end_time - start_time))

Epoch 0: 7489 / 10000
Epoch 1: 8452 / 10000
Epoch 2: 8715 / 10000
Epoch 3: 8854 / 10000
Epoch 4: 8946 / 10000
Epoch 5: 9001 / 10000
Epoch 6: 9043 / 10000
Epoch 7: 9075 / 10000
Epoch 8: 9113 / 10000
Epoch 9: 9146 / 10000
Epoch 10: 9163 / 10000
Epoch 11: 9175 / 10000
Epoch 12: 9197 / 10000
Epoch 13: 9199 / 10000
Epoch 14: 9228 / 10000
Duration: 0:01:47.721579


In [ ]:
training_data, validation_data, test_data = load_data_wrapper()
#training_data,  test_data = mnist_loader.load_data_wrapper()
start_time = datetime.now()

net = Network([784, 30, 10])
net.SGD(training_data, 30, 50, 3.0, test_data=test_data)

end_time = datetime.now()
print('Duration: {}'.format(end_time - start_time))

Epoch 0: 7764 / 10000
Epoch 1: 8115 / 10000
Epoch 2: 8920 / 10000
Epoch 3: 9108 / 10000
Epoch 4: 9169 / 10000
Epoch 5: 9213 / 10000
Epoch 6: 9228 / 10000
Epoch 7: 9259 / 10000
Epoch 8: 9278 / 10000
Epoch 9: 9304 / 10000
Epoch 10: 9319 / 10000
Epoch 11: 9318 / 10000
Epoch 12: 9342 / 10000
Epoch 13: 9343 / 10000
Epoch 14: 9365 / 10000
Epoch 15: 9355 / 10000
Epoch 16: 9396 / 10000
Epoch 17: 9384 / 10000
Epoch 18: 9383 / 10000
Epoch 19: 9387 / 10000
Epoch 20: 9393 / 10000
Epoch 21: 9411 / 10000
Epoch 22: 9403 / 10000
Epoch 23: 9409 / 10000
Epoch 24: 9413 / 10000
Epoch 25: 9419 / 10000
Epoch 26: 9419 / 10000
Epoch 27: 9435 / 10000
Epoch 28: 9436 / 10000
Epoch 29: 9423 / 10000
Duration: 0:03:38.135270


In [ ]:
training_data, validation_data, test_data = load_data_wrapper()
#training_data,  test_data = mnist_loader.load_data_wrapper()
start_time = datetime.now()

net = Network([784, 10, 10])
net.SGD(training_data, 10, 30, 1.5, test_data=test_data)

end_time = datetime.now()
print('Duration: {}'.format(end_time - start_time))

Epoch 0: 7432 / 10000
Epoch 1: 8447 / 10000
Epoch 2: 8670 / 10000
Epoch 3: 8746 / 10000
Epoch 4: 8808 / 10000
Epoch 5: 8871 / 10000
Epoch 6: 8876 / 10000
Epoch 7: 8899 / 10000
Epoch 8: 8932 / 10000
Epoch 9: 8967 / 10000
Duration: 0:00:33.107510


In [ ]:
training_data, validation_data, test_data = load_data_wrapper()
#training_data,  test_data = mnist_loader.load_data_wrapper()
start_time = datetime.now()

net = Network([784, 15, 10])
net.SGD(training_data, 20, 100, 0.3, test_data=test_data)

end_time = datetime.now()
print('Duration: {}'.format(end_time - start_time))

Epoch 0: 1966 / 10000
Epoch 1: 3337 / 10000
Epoch 2: 4301 / 10000
Epoch 3: 4916 / 10000
Epoch 4: 5373 / 10000
Epoch 5: 5691 / 10000
Epoch 6: 5972 / 10000
Epoch 7: 6306 / 10000
Epoch 8: 6525 / 10000
Epoch 9: 6702 / 10000
Epoch 10: 6844 / 10000
Epoch 11: 6956 / 10000
Epoch 12: 7050 / 10000
Epoch 13: 7150 / 10000
Epoch 14: 7251 / 10000
Epoch 15: 7410 / 10000
Epoch 16: 7751 / 10000
Epoch 17: 7968 / 10000
Epoch 18: 8067 / 10000
Epoch 19: 8168 / 10000
Duration: 0:01:56.426321
